In [ ]:
import rgxlog

## I'll try to breakdown how the engine works with a couple of examples

## 1- Where all the data about the relations is stored ?
They are using sqlite3 database to store and manipulate the information, heres a brief about sqlite3 from chatGPT:
SQLite3 is a module in Python's standard library that provides a lightweight and self-contained relational database management system. It allows you to create, connect to, and manipulate SQLite databases using Python.

## 2- What specific data is stored in the db tables, and what is the structure of these tables ?
Whenever we define a relation, a corresponding table is created in the database with the same name as the relation. The number of columns in this table is equal to the number of variables in the relation i.e.len(term_list). As for the rows, they are added or deleted in response to the addition or removal of facts related to the corresponding relation.

In [ ]:
%%rgxlog
new brothers(str, str)

when we declared a new relation a table was declared for us in the database with the name as of the relation's name and with the number of cols as the number of parameters in the relation i.e len(term_list).
the names of columns in the database don't have an informative name and there is no need because from Python's perspective it doesn't understand what the relation or its values mean, it just sorts information in the way they were provided, for example brothers("Jack","Michael") from Python's perspective it just sees that "Jack" is the first term so it puts it in the col0 and "Michael" is the second so it gets added to col1.

In [ ]:
%rgxlog brothers("Kamil", "7mode")

And as described above, adding facts to a relation adds lines in the corresponding table in the db.

In [ ]:
%rgxlog brothers("Kamil", "7mode") <- True

In [ ]:
%rgxlog brothers("Kamil", "7mode") <- False

OperationalError: near ",": syntax error

In [ ]:
%%rgxlog
new parent(str ,str)
parent("bob", "greg")
parent("greg", "alice")
parent("George", "William")
parent("William", "Jack")

Entered declare_relation_table with args: (parent(str, str),) {}
Entered add_fact with args: (parent("bob", "greg"),) {}
Table: parent
  col0  col1
0  bob  greg

Entered add_fact with args: (parent("greg", "alice"),) {}
Table: parent
   col0   col1
0   bob   greg
1  greg  alice

Entered add_fact with args: (parent("George", "William"),) {}
Table: parent
     col0     col1
0     bob     greg
1    greg    alice
2  George  William

Entered add_fact with args: (parent("William", "Jack"),) {}
Table: parent
      col0     col1
0      bob     greg
1     greg    alice
2   George  William
3  William     Jack



## Now for the fun part, let's understand how rules and queries work :P

When a new rule is added, an empty table is created in the database. The reason for the table being empty is that multiple rules can be defined with the same rule head. During querying, the system iterates through the list of rules, and the query result is obtained by performing a logical OR operation on the results of these rules.
for example: <br>
    if we define the rules: <br>
        A(X,Y) <- B(X,Y) <br>
        A(X,Y) <- C(X,Y) <br> 
        and then we run the query: ?A(X,Y) <br>
        we get in the results printed all the B(X,Y) OR C(X,Y)

## Queries
In order to understand how querying works we need to bearkdown the rule into parts:
for example lets breakdown the rule: <br>
grandparent(X,Z) <- parent(X,Y), parent(Y,Z) <br>
the rule head: grandparent(X,Z)
the rule body relations: [parent(X,Y), parent(Y,Z)]

There are 4 operators in the engine that get called when breaking that rule, lets talk about each one of them.
Note: each operator yields a relation that is added to the database temporarily. (deleted when we're done querying)


### 1- operator_select(self, src_relation: Relation, constant_variables_info: Set[Tuple[int, Any, DataTypes]], *args: Any) -> Relation

The operator_select function operates on a relation, which is represented as a database table, and a set of constant variables. Its primary objective is to filter tuples from the given relation based on certain conditions and return the filtered relation.

For instance, consider the rule: brothers_of_loay(X) <- brothers(X, "Loay"). The goal is to print all the brothers of "Loay" by retrieving tuples from the "brothers" relation table in the database where "Loay" appears on the right side of the tuple.

The operator_select function serves precisely this purpose. When called with the above relation, it receives the following parameters: <br>
"Entered operator_select with args: (brothers(X, "Loay"), {(1, 'Loay', <DataTypes.string: 0>)})"

Here, constant_variables_info represents a set of tuples. Each tuple contains information about a constant variable. The first index of each tuple denotes the index of the constant variable within the relation (e.g., 1 in our case), while the second index stores the value of the constant variable (e.g., 'Loay').

Let's consider an example:

In [ ]:
%%rgxlog
brothers("Kamil", "7mode")
brothers("Kamil", "Loay")
brothers("7mode", "Loay")
brothers_of_loay(X) <- brothers(X, "Loay")
?brothers_of_loay(X)

Entered add_fact with args: (brothers("Kamil", "7mode"),) {}
Table: brothers
    col0   col1
0  Kamil  7mode

Entered add_fact with args: (brothers("Kamil", "Loay"),) {}
Table: brothers
    col0   col1
0  Kamil  7mode
1  Kamil   Loay

Entered add_fact with args: (brothers("7mode", "Loay"),) {}
Table: brothers
    col0   col1
0  Kamil  7mode
1  Kamil   Loay
2  7mode   Loay

Entered declare_relation_table with args: (brothers_of_loay(),) {}
Entered declare_relation_table with args: (__rgxlog__brothers_select0(),) {}
Table: __rgxlog__brothers_select0
    col0  col1
0  Kamil  Loay
1  7mode  Loay

Entered declare_relation_table with args: (__rgxlog__brothers_select0_project1(),) {}
Entered operator_union with args: ([__rgxlog__brothers_select0_project1(X)], None) {}
Entered declare_relation_table with args: (__rgxlog__brothers_of_loay_select2(),) {}
Table: __rgxlog__brothers_of_loay_select2
    col0
0  Kamil
1  7mode

Entered declare_relation_table with args: (__rgxlog__brothers_of_loay_sel

### 2- operator_project(self, src_relation: Relation, project_vars: List[str], *args: Any) -> Relation

The operator_project function operates on a relation, which is represented as a database table, and a list of project variables. Its primary objective is to yield a relation with the projected variables of the src_relation.

Continuing from the previous example, once the select relation table is added to the database, we observe that it consists of two columns where "Loay" appears on the right-hand side. However, the purpose of the query is to print only a single column containing the brothers of "Loay", without including the right column that holds the value "Loay".

To address this requirement, the project operator comes into play. Its primary objective is to operate on the relation resulting from the select operation and generate another relation where "Loay" no longer appears on the right-hand side.

In the given example, the project operator is invoked after the select operation has been executed: <br>
Entered operator_project at 2023-06-04 06:55:25 with args: (__rgxlog__brothers_select0(X, "Loay"), ['X']) <br>
we see that it is working on the relation that operator_select added to the data base and takes 'X' as a project variable.

so another relation table gets added to the data base which is : __rgxlog__brothers_select0_project1 <br>
and that table would look like that: <br>
Table: __rgxlog__brothers_select0_project1 <br>
    col0 <br>
0  Kamil <br>
1  7mode

### 3- operator_join(self, relations: List[Relation], *args: Any) -> Relation

The `operator_join` function operates on a list of relations and its primary objective is to generate a logical AND operator among the tables of those relations within the database. This function plays a crucial role in situations where the following rule is defined: <br> grandparent(X,Z) <- parent(X,Y), parent(Y,Z). <br> 

The purpose of creating a relation through the `operator_join` function is to establish a connection between the columns containing all possible combinations of triplets (X, Y, Z) where both parent(X,Y) and parent(Y,Z) evaluate to true. The reason for having three columns instead of two is to ensure that we have an answer for every query pertaining to the grandparent rule. With three free variables, it becomes essential to capture all possible combinations in order to provide comprehensive responses to queries. Subsequently, the columns required by a specific query can be projected at a later stage.

It should be noted that the relation generated at this point represents the logical AND operation applied to all the relations within the rule's body relation, considering the free variables involved.

To provide a more detailed explanation, the process can be described as follows:

Given the rule with the body relations parent(X,Y) and parent(Y,Z), the operator_join function performs the following steps:

Assign Temporary Names: Each relation in the rule's body is assigned a temporary name to distinguish them during the join operation. For instance, let's assign the names as follows: parent(X,Y) is assigned the name "table0" and parent(Y,Z) is assigned the name "table1".

Matching Tuple Criteria: The objective is to retrieve tuples where table0.col1 is equal to table1.col0. This condition is applied to combine the two relations and generate a new relation.

Resulting Relation: The resulting relation, known as "joined_relation," consists of three columns: table0.col0, table0.col1, and table1.col1. Notably, the reason why we don't include table1.col0 as a separate column is because it is known to be the same as table0.col1.

In [ ]:
%rgxlog grandparent(X,Z) <- parent(X,Y), parent(Y,Z)

Entered declare_relation_table with args: (grandparent(),) {}


In [ ]:
%rgxlog ?grandparent(X,Y)

Entered operator_join with args: ([parent(X, Y), parent(Y, Z)], {'Z': [(parent(Y, Z), 1)], 'X': [(parent(X, Y), 0)], 'Y': [(parent(X, Y), 1), (parent(Y, Z), 0)]}) {}
Entered declare_relation_table with args: (__rgxlog__join4(),) {}
Table: __rgxlog__join4
    col0    col1     col2
0  alice     bob     greg
1   Jack  George  William

Ask Loay about why the following tables were declared: (the tables below)
Entered declare_relation_table with args: (__rgxlog__join4_project5(),) {}
Entered operator_union with args: ([__rgxlog__join4_project5(X, Z)], None) {}
Entered declare_relation_table with args: (__rgxlog__grandparent_select6(),) {}
Table: __rgxlog__grandparent_select6
     col0   col1
0     bob  alice
1  George   Jack

Entered declare_relation_table with args: (__rgxlog__grandparent_select6_project7(),) {}
printing results for query 'grandparent(X, Y)':
   X    |   Y
--------+-------
  bob   | alice
 George | Jack



### 4- operator_union(self, relations: List[Relation], *args: Any) -> Relation
As previously mentioned, a rule can be defined multiple times as long as it shares the same rule head. To illustrate this, let's consider the following example:

Suppose we have the rules:
A(X,Y) <- B(X,Y)
A(X,Y) <- C(X,Y)

If we execute the query ?A(X,Y), the engine will produce results that include tuples from both B(X,Y) and C(X,Y) relations. 

Here's how the engine handles this scenario:

1. Rule Matching: The engine identifies all the rules that have the same rule head (A(X,Y)) as the queried relation.

2. Rule Execution: For each matched rule, the engine executes all the steps described earlier, such as assigning temporary names, performing joins, and creating intermediate relations, selecting and filtering.

3. Intermediate Relations: At this stage, we have intermediate relations generated for each rule, containing the relevant tuples based on the rule's body.

4. Operator Union: To consolidate the results from all the rules into a single unified relation, the operator union is applied. This operation combines the tuples from all the intermediate relations.

The operator union ensures that the final unified relation contains all the tuples from the individual intermediate relations, effectively merging the results obtained from different rule instances with the same rule head.

In summary, the engine processes each rule with the same rule head as the queried relation, performs the necessary steps for each rule individually, and then applies the operator union to merge the intermediate relations into a single unified relation, ensuring all relevant tuples are included in the final result.

There is an example in the next cells:

In [ ]:
%%rgxlog
new sisters(str,str)
sisters("Asmaa", "Fatima")
sisters("Zainab", "Mawada")
sisters("Arwa <3", "Arosh :P")

Entered declare_relation_table with args: (sisters(str, str),) {}
Entered add_fact with args: (sisters("Asmaa", "Fatima"),) {}
Table: sisters
    col0    col1
0  Asmaa  Fatima

Entered add_fact with args: (sisters("Zainab", "Mawada"),) {}
Table: sisters
     col0    col1
0   Asmaa  Fatima
1  Zainab  Mawada

Entered add_fact with args: (sisters("Arwa <3", "Arosh :P"),) {}
Table: sisters
      col0      col1
0    Asmaa    Fatima
1   Zainab    Mawada
2  Arwa <3  Arosh :P



In [ ]:
%rgxlog brothers_or_sisters(X,Y) <- sisters(X,Y)
%rgxlog brothers_or_sisters(X,Y) <- brothers(X,Y)

Entered declare_relation_table with args: (brothers_or_sisters(),) {}
Entered declare_relation_table with args: (brothers_or_sisters(),) {}


In [ ]:
%rgxlog ?brothers_or_sisters(X,Y)

Entered declare_relation_table with args: (__rgxlog__sisters_project8(),) {}
Entered declare_relation_table with args: (__rgxlog__brothers_project9(),) {}
Entered operator_union with args: ([__rgxlog__sisters_project8(X, Y), __rgxlog__brothers_project9(X, Y)], None) {}
Entered declare_relation_table with args: (__rgxlog__union10(),) {}
Entered declare_relation_table with args: (__rgxlog__brothers_or_sisters_select11(),) {}
Table: __rgxlog__brothers_or_sisters_select11
      col0      col1
0    7mode      Loay
1  Arwa <3  Arosh :P
2    Asmaa    Fatima
3    Kamil     7mode
4    Kamil      Loay
5   Zainab    Mawada

Entered declare_relation_table with args: (__rgxlog__brothers_or_sisters_select11_project12(),) {}
printing results for query 'brothers_or_sisters(X, Y)':
    X    |    Y
---------+----------
  7mode  |   Loay
 Arwa <3 | Arosh :P
  Asmaa  |  Fatima
  Kamil  |  7mode
  Kamil  |   Loay
 Zainab  |  Mawada

